In [4]:
from tqdm import tqdm
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("blue_score.ipynb"), "..")))
from transformers_models.marian.marianMT import MarianMt

In [ ]:
def retrieve_data(filenames):

    english_cleaned = []
    german_cleaned = []
    base_path = os.path.join(os.path.dirname("blue_score.ipynb"), "..", "data")
    en_path = os.path.join(base_path, filenames[0])
    de_path = os.path.join(base_path, filenames[1])

    with open(en_path, mode = "r", encoding = "utf-8") as f_en, \
        open(de_path, mode = "r", encoding = "utf-8") as f_de:
        for en, de in tqdm(zip(f_en.readlines(), f_de.readlines())):
            english_cleaned.append(en[:-1])
            german_cleaned.append(de[:-1])

    return (english_cleaned, german_cleaned)


In [9]:
def translated(n, model):
    return model.translate_text(n)

In [10]:
cleaned = retrieve_data(["test.en", "test.de"])
english_cleaned_to_translate = cleaned[0]
german_cleaned_reference = cleaned[1]

model = MarianMt("Helsinki-NLP/opus-mt-en-de")
translations = [translated(n, model) for n in english_cleaned_to_translate]

print(translations)


FileNotFoundError: [Errno 2] No such file or directory: '/data/test.en'